In [1]:
import geopandas as gpd
import pandas as pd
import ntpath
import glob

In [2]:
# Add the python path to the folder containing useful packages.
import sys
sys.path.insert(0, "../../packages/")

In [3]:
from visualization.TsIP import TsIP

In [7]:
a = pd.read_csv("../../../Code/Data Sources/Food Consumption Score (FCS)/time-series/Yemen/wfp_fcs.csv", header = [0, 1], index_col = 0)
a.index.name = "Datetime"

In [8]:
a.head()

AdminStrata,Abyan,Aden,Al Bayda,Al Dhale'e,Al Hudaydah,Al Jawf,Al Maharah,Al Mahwit,Amanat Al Asimah,Amran,...,Hadramaut,Hajjah,Ibb,Lahj,Marib,Raymah,Sa'ada,Sana'a,Shabwah,Taizz
Indicator,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,...,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS
Datetime,,,,,,,,,,,,,,,,,,,,,
2018-07-02,31.158690,16.619519,38.332669,29.194825,20.788151,22.085706,2.892308,16.815145,17.000398,20.446735,...,23.822825,28.361345,25.036668,31.762436,32.596233,54.121774,17.676022,26.917713,16.752289,26.563365
2018-07-03,32.675222,16.370603,43.292084,31.336940,20.692544,23.477196,2.913825,19.833443,17.208166,21.689014,...,23.699881,28.805448,27.102390,32.049499,33.333333,56.722689,17.134560,25.783476,18.257453,29.124005
2018-07-04,33.415597,15.089163,45.128205,33.135157,21.825051,24.864000,2.950408,19.614289,18.070801,25.611124,...,23.103187,30.993706,29.844066,33.106267,36.883683,56.306306,17.454545,25.436047,19.083081,29.427973
2018-07-05,33.531451,15.766521,43.959297,34.554309,21.481693,27.814992,2.247913,19.558566,19.021964,27.922484,...,21.596419,32.905902,29.025363,32.938828,36.897633,56.231003,17.846350,27.421759,20.349533,30.456026
2018-07-06,33.951856,16.870065,44.516521,33.878557,21.076712,28.688245,2.269780,17.608130,19.817567,28.712235,...,20.129084,32.598181,30.692766,33.837934,36.669568,57.324841,18.463057,26.547231,21.107524,31.012517


# Coordinates

In this notebook, we create *static time-series* regarding the coordinates (longitude and latitude) of each province (or sub-province) of the following countries:

- Yemen;
- Nigeria;
- Burkina Faso;
- Syria;

With the term *static*, we mean time-series that not change over time. We select a certain starting year as reference to create our time-series until another reference year.

In [4]:
# Define the path where the results arising from this analysis will be saved.
PATH_TO_SAVE_DATA = "./output_timeseries/"

In [5]:
# Define reference dates to build the time-series.
FIRST_YEAR = "2017"
LAST_YEAR = "2021"

In [6]:
# Read the shapefiles of each country.
path = "../Shape Files/shapefiles/"
all_folders = glob.glob(path + "*")

dfs = []

for folder in all_folders:
    country = ntpath.basename(folder).split(".")[0]
    all_files = glob.glob(path + country + "/*.shp")
    for filename in all_files:
        df = gpd.read_file(filename)
        df["Country"] = country
        dfs.append(df)

In [7]:
df = pd.concat(dfs, axis = 0, ignore_index = True)
df.head()

,admin,geometry,Country
0,Boucle-Du-Mouhoun,"POLYGON ((-3.18158 13.67744, -3.18109 13.67721...",Burkina Faso
1,Cascades,"POLYGON ((-5.38923 11.00086, -5.37382 10.97698...",Burkina Faso
2,Centre,"POLYGON ((-1.60508 12.67250, -1.58739 12.67057...",Burkina Faso
3,Centre-Est,"POLYGON ((-0.30646 12.55612, -0.30573 12.55588...",Burkina Faso
4,Centre-Nord,"MULTIPOLYGON (((-0.76975 12.55062, -0.77002 12...",Burkina Faso


In [8]:
def coordinates(row):
    lon = row.geometry.centroid.x
    lat = row.geometry.centroid.y
    return pd.Series([lon, lat])

In [9]:
# Change the name of adminstrata feature.
df.rename(columns = {"admin": "AdminStrata"}, inplace = True)
# Create coordinates features.
df[["Lon", "Lat"]] = df.apply(coordinates, axis = 1)
# Delete geometry feature.
df.drop(columns = "geometry", inplace = True)
df.head()

,AdminStrata,Country,Lon,Lat
0,Boucle-Du-Mouhoun,Burkina Faso,-3.488905,12.535927
1,Cascades,Burkina Faso,-4.570349,10.354484
2,Centre,Burkina Faso,-1.499961,12.325874
3,Centre-Est,Burkina Faso,-0.186052,11.607598
4,Centre-Nord,Burkina Faso,-0.973623,13.271074


In [10]:
# Create multi-index columns dataframe.
df = df.set_index(["Country", "AdminStrata"]).unstack().unstack().reorder_levels([2, 1, 0]).to_frame().sort_index(0).transpose().dropna(axis = 1)
df.columns.set_names("Indicator", level = 2, inplace = True)
df

Country          Burkina Faso                                            \
AdminStrata Boucle-Du-Mouhoun             Cascades               Centre   
Indicator                 Lat       Lon        Lat       Lon        Lat   
0                   12.535927 -3.488905  10.354484 -4.570349  12.325874   

Country                                                           ...  \
AdminStrata           Centre-Est           Centre-Nord            ...   
Indicator         Lon        Lat       Lon         Lat       Lon  ...   
0           -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   

Country          Yemen                                                         \
AdminStrata     Sa'ada                Sana'a               Shabwah              
Indicator          Lat        Lon        Lat        Lon        Lat        Lon   
0            17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   

Country                                                  
AdminStrata    Socotra                 Taizz             
Indicator          Lat        Lon        Lat        Lon  
0            12.488974  53.805239  13.416517  43.778161  

[1 rows x 190 columns]

In [11]:
# Create static time-series of the coordinates.
new_index = pd.date_range(FIRST_YEAR, LAST_YEAR, freq = "D")
df = df.loc[df.index.repeat(len(new_index))]
df.index = new_index
df.index.name = "Datetime"
df

Country          Burkina Faso                                            \
AdminStrata Boucle-Du-Mouhoun             Cascades               Centre   
Indicator                 Lat       Lon        Lat       Lon        Lat   
Datetime                                                                  
2017-01-01          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2017-01-02          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2017-01-03          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2017-01-04          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2017-01-05          12.535927 -3.488905  10.354484 -4.570349  12.325874   
...                       ...       ...        ...       ...        ...   
2020-12-28          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2020-12-29          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2020-12-30          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2020-12-31          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2021-01-01          12.535927 -3.488905  10.354484 -4.570349  12.325874   

Country                                                           ...  \
AdminStrata           Centre-Est           Centre-Nord            ...   
Indicator         Lon        Lat       Lon         Lat       Lon  ...   
Datetime                                                          ...   
2017-01-01  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2017-01-02  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2017-01-03  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2017-01-04  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2017-01-05  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
...               ...        ...       ...         ...       ...  ...   
2020-12-28  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2020-12-29  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2020-12-30  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2020-12-31  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2021-01-01  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   

Country          Yemen                                                         \
AdminStrata     Sa'ada                Sana'a               Shabwah              
Indicator          Lat        Lon        Lat        Lon        Lat        Lon   
Datetime                                                                        
2017-01-01   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2017-01-02   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2017-01-03   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2017-01-04   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2017-01-05   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
...                ...        ...        ...        ...        ...        ...   
2020-12-28   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2020-12-29   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2020-12-30   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2020-12-31   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2021-01-01   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   

Country                                                  
AdminStrata    Socotra                 Taizz             
Indicator          Lat        Lon        Lat        Lon  
Datetime                                                 
2017-01-01   12.488974  53.805239  13.416517  43.778161  
2017-01-02   12.488974  53.805239  13.416517  43.778161  
2017-01-03   12.488974  53.805239  13.416517  43.778161  
2017-01-04   12.488974  53.805239  13.416517  43.778161  
2017-01-05   12.488974  53.805239  13.416517  43.778161  
...                ...        ...       

In [12]:
# Now save the time-series of each country keeping as indeces for each time-series the own first and last index.
def save(group, name):
    country = group.name
    group = group[country]
    # Adjust time-series group.
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    # Save.
    group.to_csv(PATH_TO_SAVE_DATA + country + "/" + name + ".csv", index_label = False)

In [13]:
# Save the dataframes.
df.groupby(level = 0, axis = 1).apply(lambda x: save(x, name = "wfp_coordinates"));

In [14]:
# Plot static time-series.
TsIP(df).interactive_plot_df(matplotlib = False, style = "lines", yaxis = "Coordinates")

RadioButtons(description='Select:', options=('Time-series', 'Missing values'), value='Time-series')

Output()